## Class 11 Agenda:
  * **Null accuracy**
  * **Confusion matrix**
  * **sensitivity, specificity, accuracy**
  * **ROC curves, AUC, setting a threshold**
  * **ROC curves, AUC: All that matters is that ordering is preserved**
  * **Pipelines: Putting your entire ML workflow together**

Today we are going to talk about how to more accurately measure models for supervised classification. Because supervised classification problems are ubiquitous (will this person churn? will this ad be clicked? will this stock go up tomorrow?), being able to evaluate how well a supervised classifier works and to be able to choose what defines a "well-performing" classifier is very important.

To that end, we will also learn how to:
  * decompose the kinds of errors a trained model makes (on unseen data)
  * decide where (at what probability) to threshold a binary classifier (one that decides between two choices) given what is acceptable in terms of the kinds of errors the model is expected to make in the wild. 
  * understand confusion matrices
  
** By the end of this notebook you will:**

- Have a working conceptual understanding of key aspects of model evaluation in machine learning
- Be able to interpret key model metrics computed using scikit-learn
- Be able to to use different model metrics for model evaluation, depending on the goals of your model
- Be able to explain what an ROC curve and AUC metric are and how they should be used when evaluating classifiers and setting classifier thresholds.

Let's import everything we're going to use today, like always:

In [ ]:
from __future__ import print_function

import warnings

warnings.filterwarnings("ignore")

In [ ]:
#data handling, model creation/evaluation
import pandas as pd
import numpy as np
from sklearn.cross_validation import KFold, train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.preprocessing import LabelEncoder, StandardScaler, PolynomialFeatures
from sklearn import metrics
import scipy.stats as stats

# visualization
%matplotlib inline
import seaborn as sns

### Null Accuracy

Null accuracy measures what our expected accuracy should be if we were to **use the most frequent response (most frequent class) as our prediction for every new sample we see.** It is a baseline against which you may want to measure your classifier, especially when your classes are really unbalanced.

In class 5, when we learned about [Logistic Regression](https://en.wikipedia.org/wiki/Logistic_regression), we attempted to build a predictive model on a dataset that was fairly unbalanced (the vertebral column dataset from UCI).

Let's use that dataset again to measure null accuracy, and see whether a basic Logistic Regression model can give higher test set accuracy than this:

In [ ]:
vertebral_data = pd.read_csv("../data/vertebral_column_2_categories.dat",sep=" ", 
                             names=["pelvic_incidence","pelvic_tilt","lumbar_lordosis_angle",
                                    "sacral_slope","pelvic_radius","spondy_grade","outcome"])
vertebral_data.outcome.value_counts()
vertebral_data.outcome = (vertebral_data.outcome =="AB").astype(int)
X = vertebral_data[vertebral_data.columns.tolist()[:-1]]
y = vertebral_data.outcome

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=1)

lr = LogisticRegression(C=1e9)
lr.fit(X_train,y_train)
y_test_pred = lr.predict(X_test)

print("Test set accuracy of LR model: ",metrics.accuracy_score(y_test, y_test_pred))

**What is the null accuracy here?**

It is simply the fraction of `AB` (class 1) samples in the test set (since that is the most frequent class in the whole dataset):

In [ ]:
# compute null accuracy manually
print("Null accuracy on the test set: ",y_test.mean())

We can compute this using the scikit-learn api by creating what's called a `DummyClassifier`.

It can create a variety of dummy models based on simple statistics it is trained on.

We are going to create one where the model simply predicts the most frequent class, by passing in a `string` to the `strategy` parameter when we generate the unfitted model: 

In [ ]:
from sklearn.dummy import DummyClassifier
dumb_model = DummyClassifier(strategy='most_frequent')
dumb_model.fit(X_train, y_train)
y_dumb_class = dumb_model.predict(X_test)
print("Most frequent class dummy classifier test accuracy: ",metrics.accuracy_score(y_test, y_dumb_class))

#### Exercise Time

* Use the same dummy classifier approach and train an LR model using 10-fold cross validation and compute the accuracy score of each model.
* Do the same with 30-fold cross validation and plot both test-set accuracy distributions. Are the two distributions overlapping? If they aren't what does that mean?

In [ ]:
pass

### Confusion Matrix

In order to more fully understand the kinds of mistakes the model is making, we need to investigate what is called the **confusion matrix** of the model on unseen (test) data. The confusion matrix simply counts the number of predictions that fall into each possible prediction bucket.

So, for a 2-class classification problem, there are 4 "prediction buckets":
  * predict 0 (normal), actual 0 (normal) - called a **correct rejection/true negative**
  * predict 0 (normal), actual 1 (abnormal) <-- this is an error called a **miss/false negative**
  * predict 1 (abnormal), actual 0 (normal) <-- this is an error called a **false alarm/false positive**
  * predict 1 (abnormal), actual 1 (abnormal) - called a **hit/true positive**
  
![confusion matrix](../images/confusion_matrix.png)

Let's compute the confusion matrix on the test set for our Logistic Regression model:

In [ ]:
# confusion matrix
cm = metrics.confusion_matrix(y_test, y_test_pred)
cm

By convention in a confusion matrix, the actual categories are the rows and the predicted values are the columns. Our confusion matrix shows that on the test set we have an equal number of misses and false alarms (both values are 6).

We can break down the confusion matrix into a variety of single-value metrics that answer specific questions about how our model is expected to fare on new data:

  * **sensitivity/true positive rate(TPR)/recall:** What fraction of the "abnormal" samples in unseen data did we correctly predict? 
  $$ TPR = \frac {\sum TP}{\sum (TP+FN)}$$
  * **specificity/true negative rate(TNR):** What fraction of "normal" samples in unseen data did we correctly predict?  
  $$ TNR = \frac {\sum TN}{\sum (TN+FP)}$$
  * **precision/positive predictive value(PPV)** How frequently is our model correct when it predicts "abnormal" on new data?
  $$ PPV = \frac {\sum TP}{\sum (TP+FP)}$$
  * **negative predictive value (NPV):** How frequently is our model correct when it predicts "normal" on new data? 
  $$ NPV = \frac {\sum TN}{\sum (TN+FN)}$$
  * **accuracy (ACC):** How frequently is our model correct on all new data, regardless of class?
  $$ ACC = \frac {\sum (TN+TP)}{\sum (TN+FN+TP+FP)}$$
  * **F1 score (F1): ** The harmonic mean of precision and recall:
  $$ F1 = 2*\frac {Precision*Recall}{Precision+Recall}$$

In [ ]:
# calculate each metric by hand
print("Sensitivity/Recall (TPR): ",cm[1,1] / float(cm[1,1] + cm[1,0]))
print("Specificity (TNR): ", cm[0,0] / float(cm[0,0] + cm[0,1]))
print("Precision (PPV): ", cm[1,1] / float(cm[1,1]+cm[0,1]))
print("NPV: ", cm[0,0] / float(cm[0,0]+cm[1,0]))
print("Accuracy: ", (cm[1,1]+cm[0,0]) / float(cm.sum()))
print("F1:", metrics.f1_score(y_test,y_test_pred))

In [ ]:
# calculate some of these metrics using sklearn and the test set samples
print("Sensitivity/Recall (TPR): ",metrics.recall_score(y_test,y_test_pred))
print("Precision (PPV): ", metrics.precision_score(y_test,y_test_pred))
print("Accuracy: ", metrics.accuracy_score(y_test,y_test_pred))
print("F1:", metrics.f1_score(y_test,y_test_pred))

Or we can compute the full classification report, which will give us precision/recall per-feature:

In [ ]:
print("Classification Report:\n", metrics.classification_report(y_test,y_test_pred))

### ROC Curves and AUC

So far, we've working with the thresholded decisions a given classifier/model makes when it outputs a prediction (all we've been looking at is the predicted category of trained models). 

However, many classifiers (including both **logistic regression** and **random forest** models) can output a "confidence" associated with their prediction (this is called a **prediction probability**). 

Let's take a look at the prediction probabilites and the predictions of our original logistic regression classifier on our single test set:

In [ ]:
#lr probabilities per category for first five samples
predicted_probs_lr = lr.predict_proba(X_test).round(3)
predictions_lr = lr.predict(X_test)

print("Logistic Regression predicted probabilities for first five samples in test set:\n",predicted_probs_lr[:5])
print("Logistic Regression predictions for first five samples in test set:\n",predictions_lr[:5])
y_test_lr_df = pd.DataFrame(
    np.concatenate((
        predicted_probs_lr,predictions_lr.reshape((predictions_lr.shape[0],-1)),
        y_test.reshape((y_test.shape[0],-1))),axis=1
    ),
    columns = ["class_0","class_1","predicted","actual"])

y_test_lr_df.head()

Lets generate the same table of predicted probabilities, predictions, and actual values, for a trained random forest classifier that contains 100 trees:

In [ ]:
rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train,y_train)

predicted_probs_rf = rf.predict_proba(X_test)
predictions_rf = rf.predict(X_test)

y_test_rf_df = pd.DataFrame(
    np.concatenate((
        predicted_probs_rf,predictions_rf.reshape((predictions_rf.shape[0],-1)),
        y_test.reshape((y_test.shape[0],-1))),axis=1
    ),
    columns = ["class_0","class_1","predicted","actual"])

y_test_rf_df.head()

In both of these cases, the prediction threshold (for either class) is set to 0.5, so the class with the probability that is over 0.5 is the predicted class.

**However, 0.5 doesn't necessarily have to be the threshold we use for our classifier's decision threshold!**

For example, we might want to be VERY CERTAIN that the classifier thinks something is class 1 (for example, a likelihood of cancer presence given some test results). 

In that case, we would require that the class 1 probability is > 0.8 (for example). As a result, the ratio of TP/TN/FN/FP would change, yielding a new confusion matrix, with a new set of precision/recall/accuracy estimates.

**In general, the default probability threshold for a given classifier maximizes accuracy, but not the other metrics like precision, recall, etc.**

But, what if we varied the prediction thresholds to obtain different TP/TN/FP/FN values (different confusion matrices) across all possible threshold values for our classifier?

This is exactly what an [ROC (Receiver Operating Characteristic) curve](https://en.wikipedia.org/wiki/Receiver_operating_characteristic) does.

**An ROC Curve is a graphical plot that illustrates the performance of a binary classifier system as its discrimination threshold is systematically varied.**

Let's generate the ROC curves for both of our models:

In [ ]:
#generate lr model false positive and true positive rates
fpr_lr, tpr_lr, thresholds_lr = metrics.roc_curve(y_test, predicted_probs_lr[:,1])

#generate same for random forest model
fpr_rf, tpr_rf, thresholds_rf = metrics.roc_curve(y_test, predicted_probs_rf[:,1])

# plot LR and RF model ROC curves
sns.plt.plot(fpr_lr, tpr_lr,label="lr")
sns.plt.plot(fpr_rf, tpr_rf,label="rf")
sns.plt.xlim([0, 1])
sns.plt.ylim([0, 1.05])
sns.plt.legend(loc="lower right")
sns.plt.xlabel('False Positive Rate (1 - Specificity)')
sns.plt.ylabel('True Positive Rate (Sensitivity)')

The ROC curve allows you to balance how many FN's (misses) and FP's (false alarms) you're willing to have with respect to one of your classes.

There's a metric that allows you to quantify the overall performance of your binary classifier, regardless of the threshold you choose. This metric is called the **AUC (area under the curve)**, and is a systematic way to compare two classifiers relative to each other, across all decision thresholds. If model A has a higher AUC than model B then this means that on average, model A is a better classifier across all decision thresholds (all probabilities). 

Besides allowing you to calculate AUC, seeing the ROC curve can help you to choose a threshold that **balances sensitivity and specificity** in a way that makes sense for the particular context.

In [ ]:
# calculate AUC for lr and rf
print("LR model AUC: ",metrics.roc_auc_score(y_test, predicted_probs_lr[:,1]))
print("RF model AUC: ",metrics.roc_auc_score(y_test, predicted_probs_rf[:,1]))

Let's make this more explicit. Let's say instead of using 0.5 as the probability threshold for class 1, we want the probability threshold where the false positive rate is no higher than 12% (because additional false positives cost too much money, or time, etc.). 

Let's take a look at a graph of the ROC curve and the class 1 thresholds as a function of FPR:

In [ ]:
# plot LR and RF model ROC curves
sns.plt.plot(fpr_lr, tpr_lr,label="lr")
sns.plt.plot(fpr_lr,thresholds_lr, label="lr_thresh")
sns.plt.xlim([0, 1])
sns.plt.ylim([0, 1.05])
sns.plt.legend(loc="center")
sns.plt.xlabel('False Positive Rate (1 - Specificity)')
sns.plt.ylabel('True Positive Rate (Sensitivity) or Class 1 Threshold Probability')


It looks like the threshold that satisfies our criteria is ~0.7. Let's use this new threshold to generate our predictions and see what our new confusion matrix looks like:

In [ ]:
y_test_lr_df["predicted_07"] = (y_test_lr_df.class_1 > 0.7).astype(float)
print(y_test_lr_df.head())
print("Confusion matrix at original 0.5 threshold:\n",metrics.confusion_matrix(y_test_lr_df.actual,
                                                                      y_test_lr_df.predicted),"\n")
print("Classification Report at original 0.5 threshold:\n", metrics.classification_report(y_test_lr_df.actual,
                                                                                          y_test_lr_df.predicted),"\n")
print("Confusion matrix at 0.7 threshold:\n",metrics.confusion_matrix(y_test_lr_df.actual,
                                                                      y_test_lr_df.predicted_07),"\n")
print("Classification Report at 0.7 threshold:\n", metrics.classification_report(y_test_lr_df.actual,
                                                                                 y_test_lr_df.predicted_07))

As a result, we've now increased our precision for class 1 (we are more certain someone has a back problem, when our classifier says you have a back problem), but have also lowered our recall for class 1 (now, more people with back problems will go un-identified by our classifier). The opposite occurs for precision and recall for class 0 as a result.

**So, depending on what you want to maximize or minimize for your machine learning application, your decision threshold for your (completely trained) classifier will differ.**

**One final important point about computing the AUC, you have to use the predicted probabilities, not the class labels themselves when computing an ROC curve or AUC.**

If you use y_pred_class, it will not give you an error, rather it will interpret the ones and zeros as predicted probabilities of 100% and 0%, and thus will give you incorrect results:

In [ ]:
# calculate AUC using y_pred_class (producing incorrect results)
print("Wrong way to calculate LR model AUC: ",metrics.roc_auc_score(y_test, predictions_lr))
print("Wrong way to calculate RF model AUC: ",metrics.roc_auc_score(y_test, predictions_rf))

As a final way to compare the two models, let's see if we can look at each of their predicted probabilites on the test set as a function of the actual category (the target):

In [ ]:
# histogram of predicted probabilities grouped by actual response value for LR
y_test_lr_df.class_1.hist(by= y_test_lr_df.actual, sharex=True, sharey=True)
#same for RF
y_test_rf_df.class_1.hist(by= y_test_rf_df.actual, sharex=True, sharey=True)

Depending on what threshold we choose will affect what part of each histogram consists of misclassified examples.

Nonetheless, it is fruitful at this point to examine all of those test-set samples that each classifier was "very certain" about (had very high or very low probability) but was of the wrong category (a class 0 sample that had a very high probability or a class 1 sample with a probability very close to 0).

### All that matters for ROC/AUC is the order of the probabilities, not their values

Let's go through our vertebral data example from start to finish to demonstrate a very important point:

In [ ]:
#load in data
vertebral_data = pd.read_csv("../data/vertebral_column_2_categories.dat",sep=" ", 
                             names=["pelvic_incidence","pelvic_tilt","lumbar_lordosis_angle",
                                    "sacral_slope","pelvic_radius","spondy_grade","outcome"])

#convert outcome into binary 0/1 attribute
le = LabelEncoder()
vertebral_data.outcome = le.fit_transform(vertebral_data.outcome)
X = vertebral_data[vertebral_data.columns.tolist()[:-1]]
y = vertebral_data.outcome

#create train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=1)
#create logistic regression object
lr = LogisticRegression()
lr.fit(X_train,y_train)
y_test_pred = lr.predict(X_test)

print("Test set accuracy of default 0.5 threshold LR model: ",metrics.accuracy_score(y_test, y_test_pred))

In [ ]:
# calculate predicted probabilities for class 1
y_pred_prob1 = lr.predict_proba(X_test)[:, 1]
# show predicted probabilities in a histogram
sns.plt.hist(y_pred_prob1)

In [ ]:
# calculate AUC
metrics.roc_auc_score(y_test, y_pred_prob1)

In [ ]:
# plot ROC curve
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred_prob1)
sns.plt.plot(fpr, tpr)
sns.plt.xlim([0, 1])
sns.plt.ylim([0, 1.05])
sns.plt.xlabel('False Positive Rate (1 - Specificity)')
sns.plt.ylabel('True Positive Rate (Sensitivity)')

### First Demonstration: Take the square root of predicted probabilities (makes them all bigger, but preserve the order of probabilities)

In [ ]:
# change the predicted probabilities
y_pred_prob2 = np.sqrt(y_pred_prob1)

In [ ]:
# here are the old ones (y_pred_prob1)
print("Old predicted probs:\n",y_pred_prob1[:10].round(3))

In [ ]:
# here are the new ones (y_pred_prob2)
print("New predicted probs:\n",y_pred_prob2[:10].round(3))

In [ ]:
# you can see the histogram changed
figure = sns.plt.figure(figsize=(12,8))
figure.add_subplot(121)
sns.plt.title("Original histogram of predicted probabilities")
sns.plt.hist(y_pred_prob1)
figure.add_subplot(122)
sns.plt.title("Histogram of square root predicted probabilities")
sns.plt.hist(y_pred_prob2)

In [ ]:
# the AUC did not change
print("Old AUC: ",metrics.roc_auc_score(y_test, y_pred_prob1))
print("New AUC: ",metrics.roc_auc_score(y_test, y_pred_prob2))

In [ ]:
# the ROC curve did not change
fpr2, tpr2, thresholds2 = metrics.roc_curve(y_test, y_pred_prob2)
figure = sns.plt.figure(figsize=(12,8))
figure.add_subplot(121)
sns.plt.plot(fpr, tpr)
sns.plt.title("Original ROC Curve")
figure.add_subplot(122)
sns.plt.title("ROC Curve of sqrt probabilities")
sns.plt.plot(fpr2, tpr2)

#### Make small predicted probabilities smaller, and make big predicted probabilities bigger

In [ ]:
y_pred_prob3 = np.where(y_pred_prob1 > 0.5, np.sqrt(y_pred_prob1), y_pred_prob1**2)
# you can see these are different from y_pred_prob1 and y_pred_prob2
y_pred_prob3[:10]

In [ ]:
# the histogram changed
# you can see the histogram changed
figure = sns.plt.figure(figsize=(12,8))
figure.add_subplot(131)
sns.plt.title("Original histogram")
sns.plt.hist(y_pred_prob1)
figure.add_subplot(132)
sns.plt.title("Histogram of square root pred probs")
sns.plt.hist(y_pred_prob2)
figure.add_subplot(133)
sns.plt.title("Histogram of newest pred probs")
sns.plt.hist(y_pred_prob3)

In [ ]:
# the AUC did not change
print("Original AUC: ",metrics.roc_auc_score(y_test, y_pred_prob1))
print("Square root probs AUC: ",metrics.roc_auc_score(y_test, y_pred_prob2))
print("Newest transformed AUC: ",metrics.roc_auc_score(y_test, y_pred_prob3))

In [ ]:
fpr3, tpr3, thresholds2 = metrics.roc_curve(y_test, y_pred_prob3)
# the ROC curve did not change
figure = sns.plt.figure(figsize=(12,8))
figure.add_subplot(131)
sns.plt.plot(fpr, tpr)
sns.plt.title("Original ROC Curve")
figure.add_subplot(132)
sns.plt.title("ROC Curve of sqrt probs")
sns.plt.plot(fpr2, tpr2)
figure.add_subplot(133)
sns.plt.title("ROC Curve of newest transformed probs")
sns.plt.plot(fpr3, tpr3)

The point of all of this is, as long as the ordering of probabilities for all of your samples is maintained, the ROC will remain identical.

All that matters is the order of the predicted probabilities in predicted data, not the actual values!

### ROC/Confusion Matrix/Metric Takeaways

* For binary classifiers, you want to have the highest AUC possible. To decide between two classifiers, maximize the one that has the higher AUC.
* Once you've maximized your AUC, you set about choosing your classifier threshold based on the specifics of your problem. If you need to minimize false alarms (so maximize precision) while keeping your recall some acceptable level, then you do one thing. If you need to maximize accuracy, you do something else.
* In every case, what you're doing is generating a threshold from a specific point on the ROC curve.

## Precision Recall Curve

In [ ]:
from sklearn.metrics import precision_recall_curve

In [ ]:
precision, recall, threshold = precision_recall_curve(y_test, y_pred_prob1)

In [ ]:
threshold

In [ ]:
recall

In [ ]:
precision

In [ ]:
plt.plot(recall, precision)
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.xlim([0,1])
plt.ylim([0,1])

### Pipelines: Putting An Entire Model Together End to End

Ok, the last thing we are going to learn how to do, is how to combine every aspect creating and using a supervised machine learning model:

1. Transforming your original data (removing skew, standard scaling, encoding categorical variables as numbers)
2. Training and validating a model on that data
3. Picking parameters for a given model to optimize accuracy/precision/recall/f1 score, etc.

Let's try to see how we would do this without a pipeline. Let's get some data:

In [ ]:
columns = ["sex","length","diam","height","whole","shucked","viscera","shell","age"]
numeric_columns = columns[1:-1]
categorical_columns = columns[0]
target = columns[-1]

abalone_data = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/abalone/abalone.data",names=columns)
abalone_data.head()

Now let's preprocess it in the standard way I've shown you:

1. Let's convert the categorical column using one-hot encoding
2. Standard scale (Z-score) the numeric columns

In [ ]:
#get categorical features
#drop off last column because its unnecessary
X_categorical = pd.get_dummies(abalone_data[categorical_columns]).astype(int).ix[:,:-1]

#get and transform numeric features
X_numeric = abalone_data[numeric_columns]
X_numeric[numeric_columns] = StandardScaler().fit_transform(X_numeric)

#get outcome variable
y = abalone_data[target]

#combine transformed categorical and numeric features
X_final = pd.concat((X_numeric,X_categorical),axis=1)

And now, let's do our standard 10-fold cross-validation scoring:

In [ ]:
#create rf regressor and check 10-fold RMSE
rf = RandomForestRegressor()
cross_val_scores = np.abs(cross_val_score(rf,X_final,y,scoring = "mean_squared_error", cv=10))
rmse_cross_val_scores = np.sqrt(cross_val_scores)
print("Mean 10-fold rmse: ", np.mean(rmse_cross_val_scores))
print("Std 10-fold rmse: ", np.std(rmse_cross_val_scores))

Now, we are going to do the same thing using Scikit-learn's pipeline feature. First, we are going to have a class that allows us to subselect columns that we want to work with:

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class ItemSelector(BaseEstimator, TransformerMixin):
    """For data grouped by feature, select subset of data at a provided key.

    The data is expected to be stored in a 2D data structure, where the first
    index is over features and the second is over samples.  i.e.

    >> len(data[key]) == n_samples

    Please note that this is the opposite convention to sklearn feature
    matrixes (where the first index corresponds to sample).

    ItemSelector only requires that the collection implement getitem
    (data[key]).  Examples include: a dict of lists, 2D numpy array, Pandas
    DataFrame, numpy record array, etc.

    >> data = {'a': [1, 5, 2, 5, 2, 8],
               'b': [9, 4, 1, 4, 1, 3]}
    >> ds = ItemSelector(key='a')
    >> data['a'] == ds.transform(data)

    ItemSelector is not designed to handle data grouped by sample.  (e.g. a
    list of dicts).  If your data is structured this way, consider a
    transformer along the lines of `sklearn.feature_extraction.DictVectorizer`.

    Parameters
    ----------
    key : hashable, required
        The key corresponding to the desired value in a mappable.
    """
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]

Now, we are going to make the full pipeline, from start to finish, for the entire dataset:

In [ ]:
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import OneHotEncoder

#encode the categorical column from strings to ints
le = LabelEncoder()
abalone_data["sex_encoded"] = abalone_data[[categorical_columns]].apply(le.fit_transform)

#extract the y
y = abalone_data.age

#create the feature union for the features
X_transformed_pipe = FeatureUnion(
        transformer_list=[
            # Pipeline for one hot encoding categorical column
            ('sexes', Pipeline([
                ('selector', ItemSelector(key=["sex_encoded"])),
                ('encoder', OneHotEncoder())                    
            ])),
            # Pipeline for pulling out numeric features and scaling them
            ('numeric', Pipeline([
                ('selector', ItemSelector(key=numeric_columns)),
                #('polyfeatures', PolynomialFeatures(degree=2,interaction_only=True)),
                ('scaler', StandardScaler()),
            ]))])
#create the full final pipeline
full_pipeline = Pipeline([("all_features",X_transformed_pipe),("rf_regressor",RandomForestRegressor(n_estimators=100))])

And now let's run the whole pipe through the `cross_val_score` object:

In [ ]:
#pass the pipeline directly into cross_val_score
cross_val_scores = np.abs(cross_val_score(full_pipeline,abalone_data,y,cv=10,scoring="mean_squared_error"))
rmse_cross_val_scores = np.sqrt(cross_val_scores)
print("Mean 10-fold rmse: ", np.mean(rmse_cross_val_scores))
print("Std 10-fold rmse: ", np.std(rmse_cross_val_scores))

#### Exercise Time!

* Change the pipeline to perform PCA and keep only the first 6 components on the complete feature pipe (after standard scaling numeric features and encoding the categorical feature)

In [ ]:
pass

Now let's do this on a slightly more involved example, where we will have to do some imputation (filling in of missing values).

Here the process will be as follows:

1. Encode categorical string columns as numbers using `LabelEncoder`
2. Impute missing categorical values (marked with 0 after encoding) with most frequent category using `Imputer`
3. One-hot encode the categorical columns using `OneHotEncoder`
4. Impute missing numerical values using the median value of each column using `Imputer`
5. Z-score/standardize each numerica column using `StandardScaler`
6. Combine both collections of columns (one-hot encoded categorical columns and standardized numeric columns) using `FeatureUnion`
7. Pass the whole collection to a `RandomForestClassifier` to build a Random Forest classification model.
8. Use `cross_val_score` with 10-fold cross-validation on the entire pipeline.

Ready? Let's start by loading in the data:

In [ ]:
kidney_columns = ["age","bp","sg","al","su","rbc","pc","pcc","ba","bgr","bu","sc","sod","pot","hemo","pcv","wc","rc","htn","dm","cad","appet","pe","ane","class"]
kidney_data = pd.read_csv("../data/chronic_kidney_disease.csv",header=None,na_values="?",names=kidney_columns)
kidney_data.head()

Let's rearrange the columns so that the numeric columns are together, followed by all of the categorical columns:

In [ ]:
#rearrange kidney columns as before
kidney_columns = kidney_columns[:5]+kidney_columns[9:18]+kidney_columns[5:9]+kidney_columns[18:]
kidney_data = kidney_data[kidney_columns]
kidney_data.head()

Let's encode the strings as numbers:

In [ ]:
kidney_columns[14:]

In [ ]:
kidney_data.info()

In [ ]:
#convert strings to numbers
le = LabelEncoder()
kidney_data[kidney_columns[14:]] = kidney_data[kidney_columns[14:]].fillna("").apply(le.fit_transform)
#get the X and y
X = kidney_data[kidney_columns[:-1]]
y = kidney_data["class"]
kidney_data.head()

And here is the code for the entire pipeline:

In [ ]:
from sklearn.preprocessing import Imputer

X_transformed_pipe = FeatureUnion(
        transformer_list=[
            # Pipeline for filling in missing values, one hot encoding all categorical columns
            ('categoricals', Pipeline([
                ('selector', ItemSelector(key=kidney_columns[14:-1])),
                ('imputer', Imputer(missing_values=0,strategy="most_frequent",axis=0)),
                ('encoder', OneHotEncoder())                    
            ])),
            # Pipeline for pulling out numeric features, filling in missing values, and scaling them
            ('numeric', Pipeline([
                ('selector', ItemSelector(key=kidney_columns[:14])),
                ('imputer', Imputer(strategy="median",axis=0)),
                ('scaler', StandardScaler()),
            ]))])

full_pipeline = Pipeline([("all_features",X_transformed_pipe),("rf_classifier",RandomForestClassifier())])

In [ ]:
cross_val_score(full_pipeline,X,y,cv=10)

Each pipeline object contains a sequence of steps, which are stored in a list. Each step is a tuple, where the first element is the name you gave the given step, and the second element is the transformation or model you are applying at that step:

In [ ]:
full_pipeline.steps

Let's take a look at a few steps:

In [ ]:
full_pipeline.steps[1][1]

In [ ]:
print("The first step in the pipeline:\n",full_pipeline.steps[0])
print()
print("The second step in the pipeline: \n", full_pipeline.steps[1])
print()
print("The second step's transformation/model:\n", full_pipeline.steps[1][1])
print("Since we know this is a random forest model, lets try to get the models feature importances:\n",full_pipeline.steps[1][1].feature_importances_)

Remember, in order to be able to get feature importances or coefficients of a given model, it needs to be trained first. Just like any other transformation in sklearn, you can fit a pipeline by calling its `fit` method:

In [ ]:
full_pipeline.fit(X,y)

Now that it's been fit, we can extract the feature importances as we wanted:

In [ ]:
full_pipeline.steps[1][1].feature_importances_.round(3)

What's really great about pipelines is that you can even put them into `GridSearchCV` methods, and search across parameters to tune your models. To do so, create a dictionary entry in `param_grid` that names the step (which you named earlier) and parameters you want to test:

In [ ]:
# using GridSearchCV with Pipeline
from sklearn.grid_search import GridSearchCV
estimators_range = [20,50,100]
param_grid = dict(rf_classifier__n_estimators=estimators_range)
grid = GridSearchCV(full_pipeline, param_grid, cv=20, scoring='accuracy',n_jobs=-1)
grid.fit(X, y)
print("Best cross-validated accuracy: ",grid.best_score_)
print("Best parameter found: ",grid.best_params_)
print("Fitted_model: ",grid.best_estimator_.steps[1][1])

#### Exercise Time!

* add a PCA transformation step before training the classifier
* search over the number of PCA components to keep using `GridSearchCV` (test whether to keep the first 5,10, or all components)

In [ ]:
pass